In [2]:
import pandas as pd
import talib as ta
import matplotlib.pyplot as plt


import lightgbm as lgb
from workflow import main_indicators,Larry_completo_teste,Larry_completo
from sklearn.model_selection import train_test_split
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")

In [4]:
for timeframe in [15]:
    for tempo_fechar in [-5,-10]:
        ativo = f'win_{timeframe}_full'
        colnames = ['Data','Open','High','Low','Close','Vol']
        df = pd.read_csv(f'./Dados/{ativo}.csv', names = colnames, skiprows=1)
        #df = pd.read_csv(f'./Dados/{ativo}.csv', names = colnames, sep='\t')
        df = df[df['Data'] > '2022-01-01']

        #df = main_indicators(df)

        df_acao = Larry_completo(df, tempo_fechar=tempo_fechar, excluir_seguidos = True, acao = 'all', x_stop = 2)

        #df_acao = df_acao[df_acao['tend'] == 1]

        compra = df_acao[df_acao['acao'] =='call']['resultado_binario'].sum()/df_acao[df_acao['acao'] =='call'].shape[0]
        venda = df_acao[df_acao['acao'] =='sell']['resultado_binario'].sum()/df_acao[df_acao['acao'] =='sell'].shape[0]

        compra_result = df_acao[df_acao['acao'] =='call']['resultado_valor'].sum()
        venda_result = df_acao[df_acao['acao'] =='sell']['resultado_valor'].sum()

        print(f'Timeframe: {timeframe} - Tempo_fechar: {tempo_fechar}') 
        print('')
        print(f'Venda: {venda}% -- Compra: {compra}%')
        print(f'Venda: {venda_result} -- Compra: {compra_result}')

        df_acao = df_acao.drop(columns = ['MA_9_diff_value', 'MA_9_tend', 'MA_9_tend_diff', 'preco','stop', 'realizacao_simplista', 'stop_real', 'tend'])

        df_acao['Hora'] = df_acao['Data'].apply(lambda x: x[11:])
        df_acao['Hora_h'] = df_acao['Hora'].apply(lambda x: x[:2])
        df_acao['Mes'] = df_acao['Data'].apply(lambda x: x[5:7])

        #Retirar hora especifica
        #df_acao = df_acao[df_acao['Hora_h'] != '10']


        df_new_hora = df_acao[['Hora_h','resultado_binario','resultado_valor']]
        df_new_mes = df_acao[['Mes','resultado_binario','resultado_valor']]

        print('Agrupado por Hora')

        print(df_new_hora.groupby(['Hora_h']).mean())

        print(df_new_hora.groupby(['Hora_h']).sum())

        print('Agrupado por Mes')

        print(df_new_mes.groupby(['Mes']).mean())
        
        print(df_new_mes.groupby(['Mes']).sum())

        print('')
        print('---------------------------------------------------')



Timeframe: 15 - Tempo_fechar: -5

Venda: 0.4429530201342282% -- Compra: 0.5035211267605634%
Venda: -1600.0 -- Compra: 22315.0
Agrupado por Hora
        resultado_binario  resultado_valor
Hora_h                                    
09               0.457143        71.214286
10               0.527778        16.527778
11               0.460317        76.984127
12               0.466667         8.111111
13               0.534483       -24.051724
14               0.397059       -52.941176
15               0.459459       -24.054054
16               0.444444        19.166667
17               0.478261       158.550725
18               0.666667       458.333333
        resultado_binario  resultado_valor
Hora_h                                    
09                     32           4985.0
10                     38           1190.0
11                     29           4850.0
12                     21            365.0
13                     31          -1395.0
14                     27          -360

In [11]:
import MetaTrader5 as mt5
from workflow import main_indicators,Larry_simples_inverso,Larry_simples_compra,Larry_completo
from datetime import datetime
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

ativo = 'EURUSD'
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

df = pd.DataFrame(mt5.copy_rates_from_pos(ativo, mt5.TIMEFRAME_M1, 0, 99999))
df.columns = ['Data','Open','High','Low','Close','Vol','','']
df['Data'] = df['Data'].apply(lambda x: str(datetime.fromtimestamp(x)))

In [12]:
df

,Data,Open,High,Low,Close,Vol,,
0,2022-09-21 10:33:00,0.99227,0.99227,0.99208,0.99220,86,15,0
1,2022-09-21 10:34:00,0.99221,0.99236,0.99182,0.99231,96,16,0
2,2022-09-21 10:35:00,0.99231,0.99238,0.99181,0.99192,131,18,0
3,2022-09-21 10:36:00,0.99190,0.99212,0.99177,0.99201,123,17,0
4,2022-09-21 10:37:00,0.99200,0.99205,0.99192,0.99196,90,16,0
...,...,...,...,...,...,...,...,...
99994,2022-12-28 00:28:00,1.06328,1.06359,1.06326,1.06359,39,17,0
99995,2022-12-28 00:29:00,1.06362,1.06375,1.06358,1.06375,54,18,0
99996,2022-12-28 00:30:00,1.06375,1.06385,1.06358,1.06385,60,17,0
99997,2022-12-28 00:31:00,1.06386,1.06397,1.06380,1.06393,49,17,0


In [34]:
#for timeframe in [1,5,15]:
timeframe = 1
for tempo_fechar in [-10,-20,-30, -40, -50, -60]:
    ativo = 'USDJPY5'
    colnames = ['Data','Open','High','Low','Close','Vol']
    df = pd.read_csv(f'./Dados/{ativo}.csv', names = colnames, sep = '\t')

    #df = main_indicators(df)

    df_acao = Larry_completo(df, tempo_fechar=tempo_fechar, excluir_seguidos = True, acao = 'all', x_stop = 1)

    #df_acao = df_acao[df_acao['tend'] == 1]

    compra = df_acao[df_acao['acao'] =='call']['resultado_binario'].sum()/df_acao[df_acao['acao'] =='call'].shape[0]
    venda = df_acao[df_acao['acao'] =='sell']['resultado_binario'].sum()/df_acao[df_acao['acao'] =='sell'].shape[0]

    compra_result = df_acao[df_acao['acao'] =='call']['resultado_valor'].sum()
    venda_result = df_acao[df_acao['acao'] =='sell']['resultado_valor'].sum()

    print(f'Timeframe: {timeframe} - Tempo_fechar: {tempo_fechar}') 
    print('')
    print(f'Venda: {venda}% -- Compra: {compra}%')
    print(f'Venda: {venda_result} -- Compra: {compra_result}')

    df_acao = df_acao.drop(columns = ['MA_9_diff_value', 'MA_9_tend', 'MA_9_tend_diff', 'preco','stop', 'realizacao_simplista', 'stop_real', 'tend'])

    df_acao['Hora'] = df_acao['Data'].apply(lambda x: x[11:])
    df_acao['Hora_h'] = df_acao['Hora'].apply(lambda x: x[:2])
    df_acao['Mes'] = df_acao['Data'].apply(lambda x: x[5:7])

    df_new_hora = df_acao[['Hora_h','resultado_binario','resultado_valor']]
    df_new_mes = df_acao[['Mes','resultado_binario','resultado_valor']]

    print('Agrupado por Hora')

    print(df_new_hora.groupby(['Hora_h']).mean())

    print(df_new_hora.groupby(['Hora_h']).sum())

    print('Agrupado por Mes')

    print(df_new_mes.groupby(['Mes']).mean())
    
    print(df_new_mes.groupby(['Mes']).sum())

    print('')
    print('---------------------------------------------------')

Timeframe: 1 - Tempo_fechar: -10

Venda: 0.2740740740740741% -- Compra: 0.30558955528355775%
Venda: 2.094499999999897 -- Compra: 14.201999999998634
Agrupado por Hora
        resultado_binario  resultado_valor
Hora_h                                    
00               0.294686        -0.000296
01               0.299559         0.001039
02               0.281659        -0.000926
03               0.288330         0.001200
04               0.270386         0.001951
05               0.200436        -0.006167
06               0.296912         0.001258
07               0.264237        -0.000919
08               0.234483        -0.005115
09               0.266667        -0.003164
10               0.284706         0.005585
11               0.271264         0.003444
12               0.261364         0.003253
13               0.266968        -0.004355
14               0.308824        -0.000053
15               0.286713        -0.000453
16               0.336384         0.003600
17               

KeyboardInterrupt: 